# 2: Add Latitude & Longitude to Toronto Neighborhoods data

<b>Challenge:</b> Use the Geocoder package or the csv file to create the following dataframe:

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1619481600000&hmac=ZicLq42bO2OEsQl9QQfd1ksGRhIZ6PJYK0be2-dTP7U">

Let's import required libraries

In [14]:
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
#import geocoder

Let's webscrape the list of postal codes from the Wikipedia page and then create a dictionary with 3 Key fields.

In [15]:
url = rq.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'xml')

table_contents = []
table = soup.find('table')

#table

for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

        
#table_contents
        

Let's create a new dataframe called Zip_df and fix some discrepancies in Borough data

In [17]:
Zip_df=pd.DataFrame(table_contents)
Zip_df['Borough']=Zip_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#Zip_df.head()
Zip_df.shape

(103, 3)

Create get_geocode() function and then read the CSV file from the web which include Latitude & Longitude information.

In [19]:
def get_geocode(zip_code):
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

# Assign geocode values from csv file to the dataframe
geo_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
# Rename field name to match with Zip_df dataframe
geo_df.rename(columns={'Postal Code':'PostalCode'}, inplace = True)

# Merge Zip_df data to geo_df using PostalCode field
geo_merged = pd.merge(geo_df, Zip_df, on = 'PostalCode')
# geo_merged

# Arrange fields and create TO_df new dataframe
TO_df = geo_merged[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
TO_df.head() #Final call for the second notebook

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
